# Multiple Traveling Salesman Problem (_mTSP_)


### Objetivos:

- **Modelar** soluções para o problema de programação linear para _mTSP_.
- **Implementar** a solução do problema em Julia.

No _mTSP_, problema das rotas de cidades você recebe um conjunto de cidades e as distâncias entre cada par de cidades. Sua tarefa é determinar um conjunto de **m** rotas percorrida por caixeiros vianjantes que começam em uma cidade específica e seguem um percurso que visita cada cidade exatamente uma vez, garantindo que cada uma das rotas retorne à cidade inicial.

Fique a vontade para checar os [rascunhos](https://www.canva.com/design/DAFvtaC7sSk/D_y2dgRoxvZvA8Ri5RxS1Q/edit?utm_content=DAFvtaC7sSk&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton) escritos durante a modelagem do problema original.

## Variáveis do problema:


## Função Objetivo:


## Restrições:


## Implementação em Julia:

Prepare o ambiente de execução (Google Colab):

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.7" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Julia `julia -v` instalado com sucesso!"
  echo "Recarregue esta página (pressione Ctrl+R, ⌘+R ou a tecla F5) e depois"
  echo "vá para a seção 'Verificação da Instalação'."
fi

Importe os pacotes do Solver de problemas lineares:

In [ ]:
import Pkg; Pkg.add("JuMP"); Pkg.add("Cbc");Pkg.add("Arrow");Pkg.add("Plots")

Funções Utiliárias

In [ ]:
function generate_distance_matrix(n, random_seed = 2)
    rng = Random.MersenneTwister(random_seed)
    X = 100 * rand(rng, n)
    Y = 100 * rand(rng, n)
    return X, Y
end


Declare o modelo, e defina tempo limite para processamento:

In [ ]:
using JuMP, Cbc, Plots, Arrow
model = Model(Cbc.Optimizer)
set_time_limit_sec(model,30) # 30 segundos
set_optimizer_attribute(model, "LogLevel", 0)

Exemplos de entrada:

In [ ]:
m = 3

In [ ]:
m = 2

In [ ]:
# Exemplo 1:
c =
[ 0 3 5 48 48 8 ;
3 0 3 48 48 8 ;
5 3 0 72 72 48 ;
48 48 74 0 0 6 ;
48 48 74 0 0 6 ;
8 8 50 6 6 0 ]

In [ ]:
# Exemplo 2:
c =
[ 0 3 5 48 48 8 8 5 5 3 3 0 3 5 8 8 5;
3 0 3 48 48 8 8 5 5 0 0 3 0 3 8 8 5;
5 3 0 72 72 48 48 24 24 3 3 5 3 0 48 48 24;
48 48 74 0 0 6 6 12 12 48 48 48 48 74 6 6 12;
48 48 74 0 0 6 6 12 12 48 48 48 48 74 6 6 12;
8 8 50 6 6 0 0 8 8 8 8 8 8 50 0 0 8;
8 8 50 6 6 0 0 8 8 8 8 8 8 50 0 0 8;
5 5 26 12 12 8 8 0 0 5 5 5 5 26 8 8 0;
5 5 26 12 12 8 8 0 0 5 5 5 5 26 8 8 0;
3 0 3 48 48 8 8 5 5 0 0 3 0 3 8 8 5;
3 0 3 48 48 8 8 5 5 0 0 3 0 3 8 8 5;
0 3 5 48 48 8 8 5 5 3 3 0 3 5 8 8 5;
3 0 3 48 48 8 8 5 5 0 0 3 0 3 8 8 5;
5 3 0 72 72 48 48 24 24 3 3 5 3 0 48 48 24;
8 8 50 6 6 0 0 8 8 8 8 8 8 50 0 0 8;
8 8 50 6 6 0 0 8 8 8 8 8 8 50 0 0 8;
5 5 26 12 12 8 8 0 0 5 5 5 5 26 8 8 0]

Declare as variáveis:

In [ ]:
n = length(c[1,:])
@variable(model,x[i in 1:n, j in 1:n, k in 1:m; i != j],Bin)
@variable(model,0 <= u[i in 1:n, k in 1:m] <= n-1)

Declare a função objetivo:

In [ ]:
z = sum(x[i,j,k]*c[i,j] for i in 1:n, j in 1:n, k in 1:m if i!= j)
@objective(model,Min, z)

Declare as restrições do modelo:

In [ ]:
# Garante que todos os caixeiros saiam no nó inicial
@constraint(model, depart1, m == sum( x[1,j,k] for k in 1:m, j in 2:n))
# Garante que todos os caixeiros retornem ao nó inicial
@constraint(model, comeback1, m == sum( x[j,1,k] for k in 1:m, j in 2:n))
# Nova constraint
# Garante que o k-ésimo caixeiro saia do primeiro nó apenas uma vez
@constraint(model, leaving1[ k in 1:m], 1 == sum( x[1,j,k] for j in 2:n))
# Garante que o k-ésimo caixeiro volte primeiro nó apenas uma vez
@constraint(model, arriving[ k in 1:m], 1 == sum( x[j,1,k] for j in 2:n))
# Garante que o k-ésimo caixeiro chegue em um nó apenas uma vez
@constraint(model, arriving[i in 2:n], sum(x[j, i, k] for k in 1:m, j in 1:n if i != j) == 1)
# Garante que o k-ésimo caixeiro saia de um nó apenas uma vez
@constraint(model, leaving[i in 2:n], sum(x[i, j, k] for k in 1:m, j in 1:n if i != j) == 1)
# Evita subrotas
@constraint(model,subtourelim[i in 2:n, j in 2:n, k in 1:m ; i!= j], u[i,k] - u[j,k] + n*x[i,j,k] <= n-1)
# Garante que todos os caixeiros retornem ao nó inicial
@constraint(model, grantCycle[j in 1:n, k in 1:m], sum( x[i,j,k] for i in 1:n if i != j) == sum( x[j,i,k] for i in 1:n if i != j))





Imprima detalhes do modelo:

In [ ]:
print(model)

Otimize o modelo:

In [ ]:
optimize!(model)

Verifique os resultados:

In [ ]:
plot = Plots.plot()
cores = [:blue, :red, :green, :purple, :orange, :cyan]
# Grafo em Hexagono ( válido apenas para c = 6)
#X = [0, 10, 10, 0, -10, -10]
#Y = [20, 10, -10, -20, -10, 10]
X, Y = generate_distance_matrix(n,4)
min_value = objective_value(model)
Plots.annotate!(maximum(X), maximum(Y) , text("Objective Value: $min_value", :right))

for i in 1:n
    Plots.annotate!([X[i]], [Y[i]], "$i")
end

for i in 1:n, j in 1:n, k in 1:m
    if i!= j && value(x[i,j,k]) == 1
    quiver!([X[i]], [Y[i]], quiver = ([X[j] - X[i]], [Y[j] - Y[i]]), color = cores[k])
end
end
plot

---

[Fernando Schettini](https://linktr.ee/fernandoschett) <br/>
Intern at the SENAI-CIMATEC Supercomputing Center [CS2I](https://www.senaicimatec.com.br/). <br/>
[João Bernardino]() <br/>
Computer Science Student at UFBA.<br/>